In [ ]:
import polars as pl
import databpy as db
import numpy as np
from io import StringIO

# Updated JSON data
json_file = StringIO("""
{
  "Dino": [
    [55.3846, 97.1795, 0.0],
    [51.5385, 96.0256, 0.0]
  ],
  "Star": [
    [58.2136, 91.8819, 0.0],
    [58.1961, 92.215, 0.0]
  ],
  "Is_Visible": [
    [true],
    [false]
  ],
  "Intensity": [
    [10],
    [20]
  ],
  "My Strings": [
    ["A"],
    ["B"]
  ]
}
""")

# Load data into Polars DataFrame
df = pl.read_json(json_file)
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
df = df.explode(columns_to_explode)

# Create Bob object
vertices = np.zeros((len(df), 3), dtype=np.float32)
bob = db.create_bob(vertices, name="DinoStar")

# Store named attributes
for col in df.columns:
    
    # Skip columns that are string-typed
    if df[col].dtype == pl.List(pl.Utf8):
        print(f"Skipping column '{col}' as it contains string data.")
        continue

    data = df[col].to_numpy()
    if df[col].dtype == pl.List:
        data = np.vstack(data)
    bob.store_named_attribute(data, col)

# Check one of the named attributes
bob.named_attribute("Dino")

In [4]:
import polars as pl
import databpy as db
from io import StringIO
import numpy as np

json_file = StringIO(
    """
{
  "Star": [
    [58.2136, 91.8819, 0.0],
    [58.1961, 92.215, 0.0]
  ],
  "Is_Visible": [
    [true],
    [false]
  ],
  "Intensity": [
    [10],
    [20]
  ]
}
"""
)

df = pl.read_json(json_file)
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
df = df.explode(columns_to_explode)

vertices = np.zeros((len(df), 3), dtype=np.float32)
bob = db.create_bob(vertices, name="Hello JSON")

for col in df.columns:
    data = np.vstack(df.get_column(col).to_numpy())
    bob.store_named_attribute(data, col)

print(bob.named_attribute("Star"))
print(bob.named_attribute("Is_Visible"))
print(bob.named_attribute("Intensity"))

[[58.21360016 91.88189697  0.        ]
 [58.19609833 92.21499634  0.        ]]
[ True False]
[10 20]


In [6]:
import polars as pl
from io import StringIO

json_file = StringIO(
    """
{
  "Star": [
    [58.2136, 91.8819, 0.0],
    [58.1961, 92.215, 0.0]
  ],
  "Is_Visible": [
    [true],
    [false]
  ],
  "Intensity": [
    [10],
    [20]
  ]
}
"""
)

df = pl.read_json(json_file)
columns_to_explode = [col for col in df.columns if df[col].dtype == pl.List(pl.List)]
df2 = df.explode(columns_to_explode)
print(df)
print(df2)

shape: (1, 3)
┌─────────────────────────────────┬───────────────────┬─────────────────┐
│ Star                            ┆ Is_Visible        ┆ Intensity       │
│ ---                             ┆ ---               ┆ ---             │
│ list[list[f64]]                 ┆ list[list[bool]]  ┆ list[list[i64]] │
╞═════════════════════════════════╪═══════════════════╪═════════════════╡
│ [[58.2136, 91.8819, 0.0], [58.… ┆ [[true], [false]] ┆ [[10], [20]]    │
└─────────────────────────────────┴───────────────────┴─────────────────┘
shape: (2, 3)
┌─────────────────────────┬────────────┬───────────┐
│ Star                    ┆ Is_Visible ┆ Intensity │
│ ---                     ┆ ---        ┆ ---       │
│ list[f64]               ┆ list[bool] ┆ list[i64] │
╞═════════════════════════╪════════════╪═══════════╡
│ [58.2136, 91.8819, 0.0] ┆ [true]     ┆ [10]      │
│ [58.1961, 92.215, 0.0]  ┆ [false]    ┆ [20]      │
└─────────────────────────┴────────────┴───────────┘
